In [1]:
import tensorflow as tf
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="1"


In [2]:
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2.3.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
# TODO : 이미지들을 전체 읽어들인 후 60Fps 의 동영상으로 저장하기.
# 이를 Android 에서 받아서 추론에 사용하기 위함.

import os
import random
import cv2
from tqdm import tqdm

file_path = '/home/yousungchoi/projects/project_coordination/workspace/training_demo/images/inference_original_image'
file_names = os.listdir(file_path)

elapsed = []
obj_cnted_more = []
obj_not_cnted = []
obj_cnt = 0
total_obj_cnt = 0

# setting for Video output
video_output = '/home/yousungchoi/projects/project_coordination/workspace/training_demo/images/inference_original_image/output_1st_720x1440_320320_model.mp4'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(video_output, fourcc, 10.0, (720, 1440))
# random.shuffle(file_names)
file_names.sort()
for i, file in enumerate(tqdm(file_names)):
    if file.endswith('jpg'):
        image_path = os.path.join(file_path, file)
        image_cv2 = cv2.imread(image_path, cv2.IMREAD_COLOR)
        image_np = cv2.cvtColor(image_cv2, cv2.COLOR_BGR2RGB)
        input_tensor = np.expand_dims(image_np, 0)

        start_time = time.time()
        detections = detect_fn(input_tensor)
        end_time = time.time()
        elapsed.append(end_time - start_time)

        plt.rcParams['figure.figsize'] = [10, 10]
        label_id_offset = 1
        image_np_with_detections = image_cv2.copy()
        viz_utils.visualize_boxes_and_labels_on_image_array(
              image_np_with_detections,
              detections['detection_boxes'][0].numpy(),
              detections['detection_classes'][0].numpy().astype(np.int32),
              detections['detection_scores'][0].numpy(),
              category_index,
              use_normalized_coordinates=True,
              max_boxes_to_draw=200,
              min_score_thresh=.60,
              agnostic_mode=False)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        obj_cnt = len(np.where(detections['detection_scores'][0].numpy() > 0.6)[0])
        total_obj_cnt += obj_cnt
    
        if obj_cnt == 0 :
            obj_not_cnted.append(file)
#             print('No Count!!')
        elif obj_cnt > 1:
            obj_cnted_more.append(file)
#             print('Count more then 2!!')
            
        cv2.putText(image_np_with_detections, 'Counted {0:1d}'.format(total_obj_cnt), (20,20), cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,0,0),2,cv2.LINE_AA)
        cv2.putText(image_np_with_detections, 'noCount {0:1d}'.format(len(obj_not_cnted)), (180,20), cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,0,0),2,cv2.LINE_AA)
        cv2.putText(image_np_with_detections, 'Dbl Cnt {0:1d}'.format(len(obj_cnted_more)), (20,50), cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,0,0),2,cv2.LINE_AA)
        
        if i == len(file_names)-5:
            print(image_np_with_detections.shape)
            plt.imshow(image_np_with_detections)
#         elif i == 1001:
#             break

        out.write(image_np_with_detections)
        
out.release()

mean_elapsed = sum(elapsed) / float(len(elapsed))
print('Elapsed time: ' + str(mean_elapsed) + ' second per image')

print('Total obj_cnt : ', total_obj_cnt)
print('obj_not_cnted : ', len(obj_not_cnted))
print(obj_not_cnted)
print('obj_cnt_more : ', len(obj_cnted_more))
print(obj_cnted_more)


# print(detections.keys())